In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [1]:
#数据预处理，拆成pair<question,answer>
with open('../chinese_data/question_answer.txt',"r") as f:    #设置文件对象
    data=f.readlines()
    print(len(data))
    after_deal=[]
    for line in data:
        line=line.strip("\n")
        after_deal.append(line.split("\t"))
        
for line in after_deal[:20]:
    print(line)  

100000
['538405926243 买 二份 有没有 少点 呀', '亲亲 真的 不好意思 我们 已经 是 优惠价 了 呢 小本生意 请亲 谅解']
['那 就 等 你们 处理 喽', '好 的 亲退 了']
['那 我 不 喜欢', '颜色 的话 一般 茶刀 茶针 和 二合一 的话 都 是 红木 檀 和 黑木 檀 哦']
['不是 免 运费', '本店 茶具 订单 满 99 包邮除 宁夏 青海 内蒙古 海南 新疆 西藏 满 39 包邮']
['好吃 吗', '好吃 的']
['为什么 迟迟 不 给 我 发货', '实在 抱歉 呢 由于 订单 量 大 您 的 订单 本来 安排 今天 发货 的 呢']
['对 谢谢', '小店 尽快 给 您 发出 哦']
['3 组送 什么', '拍 2 组送 2 包 湿巾 3 组 也 是 2 包']
['那 我 马上 拍', '记得 勾选 优惠券 哦']
['每 一样 都 要点 要 二百个', '单个 的话 价格 都 是 最低 了 哦 都 是 亏本 促销 的 只是 为了 前期 冲 销量 的 548881602868547107612966547393486364547259785739 这款 单买 可以 再 便宜 鲜肉 蜜枣 豆沙 最低 53 蛋黄 肉 粽 最低 63 元 哦']
['百世', '好 的']
['⊙ o ⊙ 哦 好 的', '优惠券 有效期 至 8 月 31 日 谢谢 客官 支持 小店 哦']
['目前 我 只有 这些', '亲 只有 这些 齐全 的 才能 开']
['可是 这个 没有 到 啊', '应该 这 两天 会到 的 这个 会 联系 下 快递']
['单 怎么 下', '您 提交 以后 哪里 可以 修改 数量 的']
['吃 起来 和 新鲜 的 有 很大 差别 嘛 保鲜', '吃 起来 也 很 松脆']
['杭州 桐庐 几天 到', '一般 1 - 2 天']
['别 给 我 像 发货 一样 的 慢', '不会 的 呢']
['我 买 的 东西 发货 了 没 怎么 看不见 物流', '亲亲 实在 抱歉 仓库 那边 说 配货 的 时候 椒盐 味纸 皮 核桃 缺货 了 呢 新 的 一批 明天 才 到货 哦 到货 后 第一 时间 给 您 打包 哦 亲亲 不要 着急 哦']
['嗯 呢 要 不到 不了 啊',

In [3]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [4]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [7]:
#构造词典
voc = Voc('communicate')
pairs = after_deal
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("counted words", voc.num_words)

counted words 21393


In [8]:
#去除低频词
MIN_COUNT = 3 
voc.trim(MIN_COUNT)
keep_pairs = []
for pair in pairs:
    input_sentence = pair[0]
    output_sentence = pair[1]
    keep_input = True
    keep_output = True
    for word in input_sentence.split(' '):
        if word not in voc.word2index:
            keep_input = False
            break
    for word in output_sentence.split(' '):
        if word not in voc.word2index:
            keep_output = False
            break
    if keep_input and keep_output:
        keep_pairs.append(pair)

print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))

keep_words 9399 / 21390 = 0.4394
Trimmed from 100000 pairs to 88237, 0.8824 of total


In [9]:
#构造tensor矩阵
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)


input_variable: tensor([[ 204,  276,  255,   43,   43],
        [ 593,   44,  460,    6,    2],
        [ 255,  185,   28,  665,    0],
        [ 276,   30, 1159,  519,    0],
        [ 185,  242, 7828, 2283,    0],
        [ 150,   81,   28,  144,    0],
        [ 597,  270,  908,    2,    0],
        [  30, 1131,  356,    0,    0],
        [  22,  986, 3188,    0,    0],
        [ 142,  347,    2,    0,    0],
        [ 111,    2,    0,    0,    0],
        [ 215,    0,    0,    0,    0],
        [ 490,    0,    0,    0,    0],
        [ 276,    0,    0,    0,    0],
        [  16,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([16, 11, 10,  7,  2])
target_variable: tensor([[ 204,  369,   27, 3504,  204],
        [ 204,   16,   28,  366,  204],
        [  27,   17,  136,   17,  136],
        [  28,  593,    2,    2,    2],
        [   2,   72,    0,    0,    0],
        [   0,  436,    0,    0,    0],
        [   0,   16,    0,    0,    0],
        